In [4]:
import pandas as pd
from random import choice, randint, shuffle
from faker import Faker

faker = Faker('hr_HR')

def make_mail(s: str):
    """Funkcija koja pretvara ime u mail u formatu ime.prezime@domena, ili kao prezimeXXX@domena"""
    trans_map = {'ć': 'c', 'č': 'c', 'š': 's', 'ž': 'z', 'đ': 'd'}
    s = s.lower().translate(str.maketrans(trans_map))

    domena = choice([faker.free_email(), faker.email(safe=False), faker.company_email()])
    domena = '@' + domena.split('@')[1]

    tip = choice(['imeprez', 'prezbr'])
    if tip == 'imeprez':
        mail = '.'.join(s.split())
        mail += domena
    elif tip == 'prezbr':
        mail = s.split()[1] + str(randint(1, 100)).zfill(3)
        mail += domena
    
    return mail

def generiraj_ljude(num: int):
    """Funkcija koja generira podatke o korisnicima"""
    imena = [faker.name() for _ in range(num)]
    spolovi = [choice(('M', 'F')) for _ in range(num)]
    mailovi = map(make_mail, imena)
    adrese = [faker.address() for _ in range(num)]
    adrese, gradovi = zip(*(adr.split('\n') for adr in adrese))
    poštanski_brojevi, gradovi = zip(*(grad.split(maxsplit=1) for grad in gradovi))
    user_ids = [randint(10000, 99999) for _ in range(num * 2)]
    # Ne mogu garantirati jedinstveni ID bez da kompliciram i radim petlje,
    # pa ovdje napravim dvostruko više, eliminiram dupliće (set makne duplikate)
    # i onda samo uzmem toliko koliko mi treba
    user_ids = list(set(user_ids))[:num]  # dedupliciranje

    ljudi = {
        'ime': imena,
        'spol': spolovi,
        'email': mailovi,
        'adresa': adrese,
        'poštanski broj': poštanski_brojevi,
        'grad': gradovi,
        'user id': user_ids
    }

    return pd.DataFrame(ljudi)

def generiraj_inventuru():
    """Funkcija koja stvara popis artikala"""
    tipovi_artikla = {
        'Men': {
            "Tops": ["T-shirt", "Polo shirt", "Dress shirt", "Sweater", "Hoodie"],
            "Bottoms": ["Jean", "Chino", "Dress pant", "Short", "Jogger"],
            "Outerwear": ["Jacket", "Blazer", "Coat", "Denim jacket"],
            "Footwear": ["Sneaker", "Loafer", "Dress shoe", "Boot", "Sandal"],
            "Accessories": ["Belt", "Watch", "Tie", "Hat", "Glove"],
        },
        'Women': {
            "Tops": ["Blouse", "Tank top", "T-shirt", "Sweater", "Hoodie"],
            "Bottoms": ["Jean", "Legging", "Skirt", "Trouser", "Short"],
            "Dresses & Jumpsuits": ["Casual dress", "Evening gown", "Jumpsuit", "Romper"],
            "Outerwear": ["Coat", "Jacket", "Blazer", "Trench coat"],
            "Footwear": ["Heel", "Flat", "Boot", "Sneaker", "Sandal"],
            "Accessories": ["Handbag", "Jewelry", "Scarf", "Sunglass", "Belt"],
        },
    }
    BOJE = ['crvena', 'plava', 'zelena', 'crna', 'bijela']

    inventura = []
    ids = set()
    for spol, tipovi in tipovi_artikla.items():
        for tip, artikli in tipovi.items():
            for artikl in artikli:
                num = randint(1, 4)
                ids = [str(randint(10000, 99999)) for _ in range(num)]
                shuffle(BOJE)
                boje = BOJE[:num]
                for id, boja in zip(ids, boje):
                    inventura.append({'spol': spol, 'tip': tip, 'artikl': artikl, 'id': id, 'boja': boja, 'cijena': randint(30, 50)})
    return pd.DataFrame(inventura)

def distribucija(težine: list[int] | dict[int, int]):
    """Funkcija koja stvara nejednaku raspodjelu. randint(1, 2) daje 50-50 šansu za oba,
    a ovako mogu napraviti da je šansa, recimo, 80-20. Funkcija prima ili listu ili dict:
    [80, 20] - 80% za 1, 20% za 2
    {3: 80, 5: 20} - 80% za 3, 20% za 5"""
    if type(težine) is list:
        težine = {idx + 1: val for idx, val in enumerate(težine)}

    suma = sum(težine.values())
    x = randint(1, suma)
    for idx, key in enumerate(težine.keys()):
        parcijalna_suma = sum(list(težine.values())[:idx + 1])
        if x < parcijalna_suma:
            break
    return key

def generiraj_narudžbu(inventura):
    """Funkcija koja generira narudžbu. Rezultat le lista s brojem narudžbe, brojem artikla i količinom artikla"""
    id_narudžbe = randint(10000, 99999)
    narudžba = []
    num_art = distribucija([50, 25, 15, 10])
    for _ in range(num_art):
        num = distribucija([85, 15])
        artikl = inventura.sample(n=1).to_dict(orient='records')[0]
        while artikl['id'] in narudžba:
            artikl = inventura.sample(n=1).to_dict(orient='records')[0]
        narudžba.append({'id': id_narudžbe, 'artikl': artikl['id'], 'komada': num})
    return narudžba

def generiraj_narudžbe(num_narudžbi, inventura, korisnici):
    narudžbe = []
    sve_narudžbe = []
    for _ in range(num_narudžbi):
        korisnik = korisnici.sample(n=1).to_dict(orient='records')[0]
        narudžba = generiraj_narudžbu(inventura)
        sve_narudžbe.extend(narudžba)
        narudžbe.append({'korisnik_id': korisnik['user id'], 'narudžba_id': narudžba[0]['id']})
        
    return pd.DataFrame(narudžbe), pd.DataFrame(sve_narudžbe)


In [5]:
korisnici = generiraj_ljude(100)
inventura = generiraj_inventuru()
narudžbe_po_korisnicima, narudžbe = generiraj_narudžbe(100, inventura, korisnici)


In [6]:
korisnici.to_csv('korisnici.csv', sep=';')
inventura.to_csv('inventura.csv', sep=';')
narudžbe.to_csv('narudžbe.csv', sep=';')
narudžbe_po_korisnicima.to_csv('po_korisnicima.csv', sep=';')